In [20]:
import dash 
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import html
from dash import dcc
from dash.dependencies import Output, Input

In [16]:
def display_selected_color(color):
    if color is None:
        color = 'nothing'
    return 'You selected ' + color

In [103]:
# app instantiation
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.PULSE])

image_path = "assets/slrpEVlogo.png"

# app layout
app.layout = html.Div([
    html.Img(src=image_path, height="100px"),
    dcc.Graph(id = "daily peak time series", figure = plot_daily_peak_power())
])

# # app layout 
# app.layout = html.Div([
#     html.Img(src=image_path, height="100px"),
#     html.H1('Poverty And Equity Database',
#             style={'color': 'blue',
#                 'fontSize': '40px'}),
#     html.H2('The World Bank'),
#     dbc.Tabs([
#         dbc.Tab([
#             html.Ul([
#                 html.Li('Number of Economies: 170'),
#                 html.Li('Temporal Coverage: 1974 - 2019'),
#                 html.Li('Update Frequency: Quarterly'),
#                 html.Li('Last Updated: March 18, 2020'),
#                 html.Li([
#                 'Source: ',
#                     html.A('https://datacatalog.worldbank.org/dataset/poverty-and-equity-database', 
#                     href='https://datacatalog.worldbank.org/dataset/poverty-and-equity-database')
#                     ])
#                 ])
#             ], label='Key Facts'),
#         dbc.Tab([
#             html.Ul([
#                 html.Br(),
#                 html.Li('Book title: Interactive Dashboards and Data Apps with Plotly and Dash'),
#                 html.Li(['GitHub repo: ', 
#                     html.A('https://github.com/PacktPublishing/Interactive-Dashboards-and-Data-Apps-with-Plotly-and-Dash', 
#                         href='https://github.com/PacktPublishing/Interactive-Dashboards-and-Data-Apps-with-Plotly-and-Dash')
#                     ])
#                 ])
#             ], label="TheDuJin")
#         ])
#     ])

# # color drop down menu
# app.layout = html.Div([
#     dcc.Dropdown(id="color_dropdown",
#         options=[{'label': color, 'value': color} for color in ['blue', 'green', 'yellow']]),
#         html.Br(),
#         html.Div(id="color_output")
# ])

# callback function
@app.callback(Output('color_output', 'children'),
              Input('color_dropdown', 'value'))
def display_selected_color(color):
    if color is None:
        color = 'nothing'
    return 'You selected ' + color
              
# running the app
if __name__ == '__main__':
    app.run_server(mode = 'inline')

In [101]:
import pandas as pd
import time

original = pd.read_csv("data/slrpEV11052020-09222022.csv")

#1
pattern = r"(\[?\{'power_W':\sDecimal\(')|('timestamp':\sDecimal\(')|('\)\}?\]?)"
power_and_time = original["power"].str.replace(pattern , "" , regex = True) # remove all pattern instances 

#2
power_and_time = power_and_time.str.split(', ')
power_and_time = power_and_time.apply(lambda lst : [int(val) for val in lst]) # cast all str value to int

power_vals = power_and_time.apply(lambda x : x[::2])
time_vals = power_and_time.apply(lambda x : x[1::2])
power_vals = power_vals.explode()
time_vals = time_vals.explode()

def round_format_UNIX_time(time_value):
    """Function takes in a UNIX time value, and rounds this value up 5 minutes.
    The time value is then converted into the format year-month-day hour:minute."""
    
    # round time up 5 minutes 
    time_value = time_value // (5 * 60) * (5*60) + (5*60)
    
    # format time value
    time_value = time.strftime('%Y-%m-%d %H:%M', time.localtime(time_value))
    
    return time_value 

time_vals = time_vals.apply(round_format_UNIX_time)
temp_df = pd.DataFrame({"time" : time_vals , "power_5m" : power_vals}) 
df = original.join(temp_df)
df_power_time = df[["time" , "power_5m"]]
df_power_time = df_power_time.groupby("time").agg(sum)
df_power_time = df_power_time.sort_values(by = "time")
df_power_time.index = pd.to_datetime(df_power_time.index)
df_power_time = df_power_time.resample("5min").sum()

from sklearn.preprocessing import OneHotEncoder

# add a new column with the name of the day of the week 
df_power_time["day"] = df_power_time.index.day_name()

def ohe_day_name(data):
    """This function takes in dataframe with a column 'day', which consists of the name of the day of the week, and one-hot encodes
    the day of the week."""
    oh_enc = OneHotEncoder()
    temp = pd.DataFrame(oh_enc.fit_transform(data[["day"]]).toarray() , columns = oh_enc.get_feature_names_out() , index = df_power_time.index)
    return pd.concat([df_power_time , temp] , axis = 1)

df_power_time = ohe_day_name(df_power_time)

# rename and reorder columns 
df_power_time.rename(columns = {"day_Friday":"Friday", "day_Monday":"Monday",
                                                "day_Saturday":"Saturday", "day_Sunday":"Sunday" ,
                                                "day_Thursday":"Thursday", "day_Tuesday":"Tuesday",
                                                "day_Wednesday":"Wednesday"} , inplace = True)
df_power_time = df_power_time[["power_5m", "day", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday" , "Sunday"]]
df_users = df[["userId" , "time"]]
df_users = df_users.groupby("time").agg("count")

# upsample into 5-minute bins 
df_users.index = pd.to_datetime(df_users.index)
df_users = df_users.resample("5min").sum()
df_users.rename(columns = {'userId':'Connected Users'}, inplace = True)
df_power_time = df_power_time.join(df_users)
df_24h = df_power_time.resample("24h").max()
df_24h.rename(columns = {"power_5m" : "max_power"} , inplace = True)

import plotly.graph_objects as go

def plot_daily_peak_power():
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x = df_24h.index, y = df_24h["max_power"] , name = "Daily Peak Power" , hovertext=df_24h["day"])
    )
    fig.update_layout(title = f"Peak Daily Power Demand", xaxis_title = "Time", yaxis_title="Power (W)")
    return fig